In [ ]:
%matplotlib inline
BATCH_SIZE = 128
GAMMA = 0.4
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 100
TARGET_UPDATE = 4
steps_done = 0

In [ ]:
# change the directory to environment directory 
#from google.colab import drive
#drive.mount('/content/drive')
#%cd /content/drive/MyDrive/Foraging_in_replenishing_patches-master (4)/Foraging_in_replenishing_patches-master

In [ ]:
!pip3 install -e gym-env

Restart the Kernal Now

In [ ]:
# change the directory to environment directory 
#%cd /content/drive/MyDrive/Foraging_in_replenishing_patches-master (4)/Foraging_in_replenishing_patches-master

In [ ]:
# This notebook was modified from the sample documented code available at: https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html

import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

env = gym.make('CartPole-v0').unwrapped

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Implementation of replay memory buffer
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
# Maps states values to action values 
class DQN(nn.Module):

    def __init__(self, state_size, action_size):
        super(DQN,self).__init__() ## calls __init__ method of nn.Module class
        self.fc1= nn.Linear(state_size,128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,action_size)
        
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.relu(self.fc3(x))

In [ ]:
n_actions = env.action_space.n
policy_net = DQN(10, n_actions).to(device)
target_net = DQN(10, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.RMSprop(policy_net.parameters(),lr=0.0001)
memory = ReplayMemory(10000)

In [ ]:
def select_action(state_a,eps):
  # select an action based on current state and epsilon randomness
    global steps_done
    sample = random.random()
    eps_threshold = eps
    if sample > eps_threshold:
        with torch.no_grad():
            return (torch.argmax(policy_net(state_a)).unsqueeze(0)).unsqueeze(0)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)


In [ ]:
def optimize_model():
  # optimizes the model by updating the neural net
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    state_action_values = policy_net(state_batch.reshape(128,10)).gather(1, action_batch)
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states.reshape(128,10)).max(1)[0].detach()
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    criterion = nn.MSELoss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [ ]:
def make_array(state,r):
  # takes states and reward and convert it into an array with one-hot encoded states followed by rewards 
  to_state = [0,0,0,0,0,0,0,0,0,0]
  to_state[state]=1
  to_state[9]=r/100
  to_state =reward = torch.tensor(to_state, device=device)
  return to_state 

In [ ]:
def playEnv(env,policy_net):  
  # code to finally play with the environment
    env.reset()
    s = env.present_state
    done = False
    a = int(np.argmax(policy_net(make_array(s,0)).detach().numpy()))

    cur_reward = 0

    while not done:
        out = env.step(a)
        (s_prime,t),r,done,_ = out
        cur_reward += r
        a = int(np.argmax(policy_net(make_array(s_prime,r)).detach().numpy()))
        
    return cur_reward

In [ ]:
from blocks import EnvClass
envs = EnvClass()
env = envs.getBlock(1) # change type of block address from here 

In [ ]:
num_episodes = 10000
eps = 1 
result = []
result_play = []
for i_episode in range(num_episodes):
    score = 0 
    # Initialize the environment and state
    env.reset()
    eps = max(eps * 0.995,0.1) 
    state = make_array(8,0) 
    for t in count():
        # Select and perform an action
        action = select_action(state,eps)
        next_state_, reward, done, _ = env.step(int(action[0][0]))
        reward = torch.tensor([reward], device=device)
        next_state = make_array(next_state_[0],reward)
        score+=reward

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            break
            
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
    t = playEnv(env,policy_net)
    result.append(score)
    result_play.append(t)

print('Complete')
env.close()

In [ ]:
playEnv(env,policy_net)

In [ ]:
r = np.array(result)
with open('deepqn.npy', 'wb') as f:
    np.save(f, r)

In [7]:
import numpy as np
a = np.load("/content/q_learning_ucb_B3_policy.npz")

In [8]:
len(a['arr_0'])

9